In [ ]:
!pip install pandas numpy requests pyarrow lightgbm scikit-learn tqdm


In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb


getting 7 days pairs data


In [ ]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime, timedelta
from tqdm import tqdm

PAIRS = [
    "BTCUSDT",
    "ETHUSDT",
    "SOLUSDT",
    "BNBUSDT",
    "XRPUSDT",
    "DOGEUSDT",
    "LTCUSDT",
    "ADAUSDT",
]

DAYS = 7


In [ ]:
import pandas as pd
import numpy as np
import requests
import io
import zipfile
from datetime import datetime, timedelta
from tqdm import tqdm

def fetch_aggtrades_day(symbol, date):
    date_str = date.strftime("%Y-%m-%d")
    url = (
        f"https://data.binance.vision/data/futures/um/daily/aggTrades/"
        f"{symbol}/{symbol}-aggTrades-{date_str}.zip"
    )

    r = requests.get(url)
    if r.status_code != 200:
        print(f"Missing {symbol} {date_str}")
        return None

    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_name = z.namelist()[0]
    df = pd.read_csv(z.open(csv_name))

    df["symbol"] = symbol
    df["date"] = date_str

    return df


In [ ]:
def fetch_pairs_aggtrades(pairs, days):
    all_dfs = []

    end_date = datetime.utcnow().date() - timedelta(days=1)
    start_date = end_date - timedelta(days=days)

    for symbol in pairs:
        print(f"\nFetching {symbol}")
        for i in tqdm(range(days)):
            day = start_date + timedelta(days=i)
            df_day = fetch_aggtrades_day(symbol, day)

            if df_day is not None:
                all_dfs.append(df_day)

    return pd.concat(all_dfs, ignore_index=True)


In [ ]:
df = fetch_pairs_aggtrades(PAIRS, DAYS)
print(df.shape)
df.head()



/tmp/ipython-input-2523268779.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow().date() - timedelta(days=1)



Fetching BTCUSDT


100%|██████████| 3/3 [00:04<00:00,  1.66s/it]



Fetching ETHUSDT


100%|██████████| 3/3 [00:06<00:00,  2.04s/it]


...

100%|██████████| 3/3 [00:01<00:00,  2.25it/s]



Fetching ADAUSDT


100%|██████████| 3/3 [00:02<00:00,  1.36it/s]


(11149920, 9)


,agg_trade_id,price,quantity,first_trade_id,last_trade_id,transact_time,is_buyer_maker,symbol,date
0,3045547981,91328.1,1.214,7083443865,7083443890,1767830400024,False,BTCUSDT,2026-01-08
1,3045547982,91328.2,0.011,7083443891,7083443894,1767830400024,False,BTCUSDT,2026-01-08
2,3045547983,91328.3,0.006,7083443895,7083443896,1767830400024,False,BTCUSDT,2026-01-08
3,3045547984,91328.4,0.023,7083443897,7083443898,1767830400024,False,BTCUSDT,2026-01-08
4,3045547985,91328.8,0.052,7083443899,7083443900,1767830400024,False,BTCUSDT,2026-01-08


In [ ]:
df

,agg_trade_id,price,quantity,first_trade_id,last_trade_id,transact_time,is_buyer_maker,symbol,date
0,3045547981,91328.1000,1.214,7083443865,7083443890,1767830400024,False,BTCUSDT,2026-01-08
1,3045547982,91328.2000,0.011,7083443891,7083443894,1767830400024,False,BTCUSDT,2026-01-08
2,3045547983,91328.3000,0.006,7083443895,7083443896,1767830400024,False,BTCUSDT,2026-01-08
3,3045547984,91328.4000,0.023,7083443897,7083443898,1767830400024,False,BTCUSDT,2026-01-08
4,3045547985,91328.8000,0.052,7083443899,7083443900,1767830400024,False,BTCUSDT,2026-01-08
...,...,...,...,...,...,...,...,...,...
11149915,550347468,0.3883,21.000,1806562685,1806562685,1768089592295,False,ADAUSDT,2026-01-10
11149916,550347469,0.3883,100.000,1806562686,1806562686,1768089593212,False,ADAUSDT,2026-01-10
11149917,550347470,0.3883,38.000,1806562687,1806562687,1768089595251,False,ADAUSDT,2026-01-10
11149918,550347471,0.3882,32.000,1806562688,1806562688,1768089597734,True,ADAUSDT,2026-01-10


In [ ]:
print("Rows:", len(df))


Rows: 11149920


In [ ]:
print("Time span:")
print(df["transact_time"].min(), "→", df["transact_time"].max())


Time span:
1767830400013 → 1768089599919


In [ ]:
df = df.rename(columns={
    "price": "price",
    "quantity": "qty",
    "transact_time": "timestamp",
    "is_buyer_maker": "is_sell"
})

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df["price"] = df["price"].astype("float32")
df["qty"] = df["qty"].astype("float32")
df["is_sell"] = df["is_sell"].astype("int8")

df = df.sort_values("timestamp").reset_index(drop=True)


******express service lol


In [ ]:
all_decisions = []

BAR = "250ms"

for symbol in PAIRS:
    print(f"\nProcessing {symbol}")

    df_sym = df[df["symbol"] == symbol].copy()

    df_sym["timestamp"] = pd.to_datetime(df_sym["timestamp"], unit="ms")
    df_sym = df_sym.sort_values("timestamp")

    df_sym["signed_qty"] = np.where(df_sym["is_sell"], -df_sym["qty"], df_sym["qty"])

    bars = (
        df_sym
        .set_index("timestamp")
        .resample(BAR)
        .agg(
            price=("price", "last"),
            qty=("qty", "sum"),
            signed_qty=("signed_qty", "sum")
        )
        .dropna()
    )

    bars["price"] = bars["price"].ffill()
    bars = bars.reset_index()
    bars["symbol"] = symbol

    # ---------- ORDER FLOW ----------
    bars["MOI_250ms"] = bars["signed_qty"].rolling(1).sum()
    bars["MOI_1s"]    = bars["signed_qty"].rolling(4).sum()

    abs_moi = bars["MOI_1s"].abs()
    mean = abs_moi.rolling(24).mean()
    std  = abs_moi.rolling(24).std()
    bars["AggressionPersistence"] = mean / (std + 1e-6)

    bars["delta_velocity"] = bars["MOI_1s"].diff()

    price_change = bars["price"].diff().abs().clip(lower=1e-4)
    bars["absorption_raw"] = bars["qty"] / price_change

    bars["absorption_z"] = (
        (bars["absorption_raw"] - bars["absorption_raw"].rolling(500).mean()) /
        (bars["absorption_raw"].rolling(500).std() + 1e-6)
    )

    # ---------- VOL ----------
    bars["ret"] = bars["price"].pct_change()
    bars["vol_5m"] = bars["ret"].rolling(1200).std()

    bars["vol_rank"] = bars["vol_5m"].rolling(2000).rank(pct=True)
    bars["vol_regime"] = pd.cut(
        bars["vol_rank"],
        bins=[-np.inf, 0.3, 0.7, np.inf],
        labels=["LOW", "MID", "HIGH"]
    )

    # ---------- LVN ----------
    BIN_SIZE = 10
    LVN_BLOCK = 1200

    bars["price_bin"] = (bars["price"] / BIN_SIZE).round() * BIN_SIZE

    lvn_price = np.full(len(bars), np.nan)
    for i in range(0, len(bars), LVN_BLOCK):
        window = bars.iloc[i:i+LVN_BLOCK]
        if window["qty"].sum() == 0:
            continue
        vp = window.groupby("price_bin")["qty"].sum()
        lvn_price[i:i+LVN_BLOCK] = vp.idxmin()

    bars["LVN_price"] = lvn_price
    bars["dist_lvn"] = (bars["price"] - bars["LVN_price"]).abs()

    # ---------- DECISION ----------
    feature_cols = [
        "MOI_250ms",
        "MOI_1s",
        "delta_velocity",
        "AggressionPersistence",
        "absorption_z",
        "dist_lvn",
        "vol_5m"
    ]

    bars = bars.dropna(subset=feature_cols)

    bars["MOI_thresh"] = bars["MOI_1s"].rolling(2000).quantile(0.8)
    bars["LVN_thresh"] = bars["dist_lvn"].rolling(2000).quantile(0.2)

    decision_mask = (
        (bars["dist_lvn"] < bars["LVN_thresh"]) |
        (bars["absorption_z"].abs() > 1.5) |
        (bars["MOI_1s"].abs() > bars["MOI_thresh"])
    )

    df_decision_sym = bars.loc[decision_mask].copy()
    df_decision_sym["bar_idx"] = df_decision_sym.index
    all_decisions.append(df_decision_sym)

    del df_sym, bars
    import gc; gc.collect()

df_decision = pd.concat(all_decisions, ignore_index=True)
X_decision_df = df_decision[feature_cols]



Processing BTCUSDT

Processing ETHUSDT

Processing SOLUSDT

Processing BNBUSDT

Processing XRPUSDT

Processing DOGEUSDT

Processing LTCUSDT

Processing ADAUSDT


In [ ]:
# one-hot encode symbol
pair_ohe = pd.get_dummies(
    df_decision["symbol"],
    prefix="pair",
    dtype="int8"
)

# merge with feature matrix
X_decision_ohe = pd.concat(
    [X_decision_df.reset_index(drop=True),
     pair_ohe.reset_index(drop=True)],
    axis=1
)


In [ ]:
FEATURE_COLUMNS = X_decision_ohe.columns.tolist()


In [ ]:
import json
with open("feature_columns.json", "w") as f:
    json.dump(FEATURE_COLUMNS, f)


In [ ]:
from google.colab import files
files.download("feature_columns.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X_decision_ohe

,MOI_250ms,MOI_1s,delta_velocity,AggressionPersistence,absorption_z,dist_lvn,vol_5m,pair_SOLUSDT
0,-341.089996,-354.609997,-358.119997,1.175089,7.071335,2.479996,0.000063,1
1,136.740005,-202.769991,151.840006,1.175431,2.224844,2.479996,0.000063,1
2,163.589996,187.659995,163.519996,1.149872,2.795418,2.490005,0.000062,1
3,-99.589996,84.890010,-99.819996,0.788964,1.671727,2.479996,0.000062,1
4,136.740005,137.480006,236.330002,1.051259,2.388673,2.479996,0.000062,1
...,...,...,...,...,...,...,...,...
856582,-0.960000,69.579998,-0.880000,0.981810,-0.230484,4.119995,0.000063,1
856583,-256.399994,-244.919994,-260.289994,0.454215,-0.185837,4.119995,0.000063,1
856584,-0.920000,-245.409994,-0.490000,0.491164,-0.211000,4.119995,0.000063,1
856585,-0.150000,-245.359994,0.050000,0.545265,-0.222733,4.119995,0.000063,1


Model spliting and all (for 300 sec)


In [ ]:
BAR_MS = 300
HORIZON_SEC = 60
HORIZON = int(HORIZON_SEC * 1000 / BAR_MS)

X_up_low, y_up_low = [], []
X_up_mid, y_up_mid = [], []
X_up_high, y_up_high = [], []

X_down_low, y_down_low = [], []
X_down_mid, y_down_mid = [], []
X_down_high, y_down_high = [], []

for symbol in PAIRS:
    print("Labeling", symbol)

    # rebuild bars for THIS symbol
    df_sym = df[df["symbol"] == symbol].copy()
    df_sym["timestamp"] = pd.to_datetime(df_sym["timestamp"], unit="ms")
    df_sym = df_sym.sort_values("timestamp")

    df_sym["signed_qty"] = np.where(df_sym["is_sell"], -df_sym["qty"], df_sym["qty"])

    bars = (
        df_sym
        .set_index("timestamp")
        .resample("250ms")
        .agg(
            price=("price", "last"),
            qty=("qty", "sum"),
            signed_qty=("signed_qty", "sum")
        )
        .dropna()
    )

    bars["price"] = bars["price"].ffill()
    bars = bars.reset_index(drop=True)

    # compute vol_5m HERE (required)
    bars["ret"] = bars["price"].pct_change()
    bars["vol_5m"] = bars["ret"].rolling(1200).std()

    # decision points for this symbol
    dec_sym = df_decision[df_decision["symbol"] == symbol]

    for i, row in dec_sym.iterrows():
        idx = int(row["bar_idx"])
        regime = row["vol_regime"]

        if idx + HORIZON >= len(bars):
            continue

        entry = bars.loc[idx, "price"]
        vol = bars.loc[idx, "vol_5m"]

        if pd.isna(vol) or vol <= 0:
            continue

        future = bars.iloc[idx+1 : idx+HORIZON+1]

        up = (future["price"].max() - entry) / entry
        down = (entry - future["price"].min()) / entry

        if max(up, down) < 0.0002:
            continue

        features = X_decision_ohe.loc[row.name].values

        if up > down:
            score = up / vol
            if regime == "LOW":
                X_up_low.append(features); y_up_low.append(score)
            elif regime == "MID":
                X_up_mid.append(features); y_up_mid.append(score)
            else:
                X_up_high.append(features); y_up_high.append(score)
        else:
            score = down / vol
            if regime == "LOW":
                X_down_low.append(features); y_down_low.append(score)
            elif regime == "MID":
                X_down_mid.append(features); y_down_mid.append(score)
            else:
                X_down_high.append(features); y_down_high.append(score)


Labeling SOLUSDT


In [ ]:
def sanitize(y):
    upper = np.percentile(y, 99)
    y = np.clip(y, 0, upper)
    return np.log1p(y)

In [ ]:
y_up_low = sanitize(np.array(y_up_low))
y_up_mid = sanitize(np.array(y_up_mid))
y_up_high = sanitize(np.array(y_up_high))
y_down_low = sanitize(np.array(y_down_low))
y_down_mid = sanitize(np.array(y_down_mid))
y_down_high = sanitize(np.array(y_down_high))

# Also convert X lists to numpy arrays for consistency and model training
X_up_low = np.array(X_up_low)
X_up_mid = np.array(X_up_mid)
X_up_high = np.array(X_up_high)
X_down_low = np.array(X_down_low)
X_down_mid = np.array(X_down_mid)
X_down_high = np.array(X_down_high)

print("UP LOW samples:", X_up_low.shape)
print("UP MID samples:", X_up_mid.shape)
print("UP HIGH samples:", X_up_high.shape)
print("DOWN LOW samples:", X_down_low.shape)
print("DOWN MID samples:", X_down_mid.shape)
print("DOWN HIGH samples:", X_down_high.shape)

UP LOW samples: (156054, 8)
UP MID samples: (100981, 8)
UP HIGH samples: (142112, 8)
DOWN LOW samples: (159608, 8)
DOWN MID samples: (107189, 8)
DOWN HIGH samples: (150375, 8)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import numpy as np
import pandas as pd

def walk_forward_splits(n, n_splits=5):
    fold_size = n // (n_splits + 1)
    for i in range(n_splits):
        tr_end = fold_size * (i + 1)
        va_end = fold_size * (i + 2)
        yield np.arange(0, tr_end), np.arange(tr_end, va_end)

def train_and_evaluate_model(X_data, y_data, name):

    X_data = np.asarray(X_data, dtype=np.float32)
    y_data = np.asarray(y_data, dtype=np.float32)

    if len(X_data) == 0:
        print(f"No data for {name}")
        return None, None

    X_df = pd.DataFrame(X_data, columns=FEATURE_COLUMNS)

    maes = []
    models = []

    print(f"\n--- Training {name} ---")

    for fold, (tr, va) in enumerate(walk_forward_splits(len(X_df))):

        model = lgb.LGBMRegressor(
            n_estimators=800,
            max_depth=6,
            learning_rate=0.03,
            subsample=0.8,
            colsample_bytree=0.8,
            min_child_samples=30,
            objective="huber",
            alpha=0.9,
            random_state=42
        )

        model.fit(
            X_df.iloc[tr], y_data[tr],
            eval_set=[(X_df.iloc[va], y_data[va])],
            eval_metric="l1",
            callbacks=[lgb.early_stopping(50, verbose=False)],
            feature_name=FEATURE_COLUMNS
        )

        preds = model.predict(X_df.iloc[va])
        mae = mean_absolute_error(y_data[va], preds)

        print(f"{name} Fold {fold} MAE: {mae:.4f}")
        maes.append(mae)
        models.append(model)

    print(f"{name} Mean MAE: {np.mean(maes):.4f}")
    print(f"{name} Target std: {np.std(y_data):.4f}")
    print(f"{name} MAE/STD: {np.mean(maes)/np.std(y_data):.4f}")


    # Evaluate top percentiles using the last model trained
    if models:
        preds = models[-1].predict(X_df.iloc[va]) # Using 'va' from the last fold
        actual = y_data[va]
        for q in [50, 75, 90]:
            idx = preds >= np.percentile(preds, q)
            if np.sum(idx) > 0:
                print(f"{name} top {100-q}% mean actual: {actual[idx].mean():.4f}")
            else:
                print(f"{name} top {100-q}% mean actual: N/A (no predictions in this percentile)")

    return maes, models


# Train and evaluate models for each category
maes_up_low, models_up_low_300 = train_and_evaluate_model(X_up_low, y_up_low, "UP LOW")
maes_up_mid, models_up_mid_300 = train_and_evaluate_model(X_up_mid, y_up_mid, "UP MID")
maes_up_high, models_up_high_300 = train_and_evaluate_model(X_up_high, y_up_high, "UP HIGH")

maes_down_low, models_down_low_300 = train_and_evaluate_model(X_down_low, y_down_low, "DOWN LOW")
maes_down_mid, models_down_mid_300 = train_and_evaluate_model(X_down_mid, y_down_mid, "DOWN MID")
maes_down_high, models_down_high_300 = train_and_evaluate_model(X_down_high, y_down_high, "DOWN HIGH")


--- Training UP LOW ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 26009, number of used features: 7
[LightGBM] [Info] Start training from score 2.583949
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
UP LOW Fold 0 MAE: 0.3877
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003130 seconds.
You can set `force_col_wise=true` to remove the overhe

In [ ]:
import os
import joblib
os.makedirs("models", exist_ok=True)
path1="models/models_up_low_300.pkl"
joblib.dump(models_up_low_300, path1)
path2="models/models_up_mid_300.pkl"
joblib.dump(models_up_mid_300, path2)
path3="models/models_up_high_300.pkl"
joblib.dump(models_up_high_300, path3)
path4="models/models_down_low_300.pkl"
joblib.dump(models_down_low_300, path4)
path5="models/models_down_mid_300.pkl"
joblib.dump(models_down_mid_300, path5)
path6="models/models_down_high_300.pkl"
joblib.dump(models_down_high_300, path6)


['models/models_down_high_300.pkl']

In [ ]:
from google.colab import files
files.download("models/models_up_low_300.pkl")
files.download("models/models_up_mid_300.pkl")
files.download("models/models_up_high_300.pkl")
files.download("models/models_down_low_300.pkl")
files.download("models/models_down_mid_300.pkl")
files.download("models/models_down_high_300.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

model for 60 sec

In [ ]:
BAR_MS = 250
HORIZON_SEC = 60
HORIZON = int(HORIZON_SEC * 1000 / BAR_MS)

X_up_low, y_up_low = [], []
X_up_mid, y_up_mid = [], []
X_up_high, y_up_high = [], []

X_down_low, y_down_low = [], []
X_down_mid, y_down_mid = [], []
X_down_high, y_down_high = [], []

for symbol in PAIRS:
    print("Labeling", symbol)

    # rebuild bars for THIS symbol
    df_sym = df[df["symbol"] == symbol].copy()
    df_sym["timestamp"] = pd.to_datetime(df_sym["timestamp"], unit="ms")
    df_sym = df_sym.sort_values("timestamp")

    df_sym["signed_qty"] = np.where(df_sym["is_sell"], -df_sym["qty"], df_sym["qty"])

    bars = (
        df_sym
        .set_index("timestamp")
        .resample("250ms")
        .agg(
            price=("price", "last"),
            qty=("qty", "sum"),
            signed_qty=("signed_qty", "sum")
        )
        .dropna()
    )

    bars["price"] = bars["price"].ffill()
    bars = bars.reset_index(drop=True)

    # compute vol_5m HERE (required)
    bars["ret"] = bars["price"].pct_change()
    bars["vol_5m"] = bars["ret"].rolling(1200).std()

    # decision points for this symbol
    dec_sym = df_decision[df_decision["symbol"] == symbol]

    for i, row in dec_sym.iterrows():
        idx = int(row["bar_idx"])
        regime = row["vol_regime"]

        if idx + HORIZON >= len(bars):
            continue

        entry = bars.loc[idx, "price"]
        vol = bars.loc[idx, "vol_5m"]

        if pd.isna(vol) or vol <= 0:
            continue

        future = bars.iloc[idx+1 : idx+HORIZON+1]

        up = (future["price"].max() - entry) / entry
        down = (entry - future["price"].min()) / entry

        if max(up, down) < 0.0002:
            continue

        features = X_decision_ohe.loc[row.name].values

        if up > down:
            score = up / vol
            if regime == "LOW":
                X_up_low.append(features); y_up_low.append(score)
            elif regime == "MID":
                X_up_mid.append(features); y_up_mid.append(score)
            else:
                X_up_high.append(features); y_up_high.append(score)
        else:
            score = down / vol
            if regime == "LOW":
                X_down_low.append(features); y_down_low.append(score)
            elif regime == "MID":
                X_down_mid.append(features); y_down_mid.append(score)
            else:
                X_down_high.append(features); y_down_high.append(score)


In [ ]:
def sanitize(y):
    upper = np.percentile(y, 99)
    y = np.clip(y, 0, upper)
    return np.log1p(y)

In [ ]:
y_up_low = sanitize(np.array(y_up_low))
y_up_mid = sanitize(np.array(y_up_mid))
y_up_high = sanitize(np.array(y_up_high))
y_down_low = sanitize(np.array(y_down_low))
y_down_mid = sanitize(np.array(y_down_mid))
y_down_high = sanitize(np.array(y_down_high))

# Also convert X lists to numpy arrays for consistency and model training
X_up_low = np.array(X_up_low)
X_up_mid = np.array(X_up_mid)
X_up_high = np.array(X_up_high)
X_down_low = np.array(X_down_low)
X_down_mid = np.array(X_down_mid)
X_down_high = np.array(X_down_high)

print("UP LOW samples:", X_up_low.shape)
print("UP MID samples:", X_up_mid.shape)
print("UP HIGH samples:", X_up_high.shape)
print("DOWN LOW samples:", X_down_low.shape)
print("DOWN MID samples:", X_down_mid.shape)
print("DOWN HIGH samples:", X_down_high.shape)

UP LOW samples: (156054, 8)
UP MID samples: (100981, 8)
UP HIGH samples: (142112, 8)
DOWN LOW samples: (159608, 8)
DOWN MID samples: (107189, 8)
DOWN HIGH samples: (150375, 8)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import numpy as np
import pandas as pd

def walk_forward_splits(n, n_splits=5):
    fold_size = n // (n_splits + 1)
    for i in range(n_splits):
        tr_end = fold_size * (i + 1)
        va_end = fold_size * (i + 2)
        yield np.arange(0, tr_end), np.arange(tr_end, va_end)

def train_and_evaluate_model(X_data, y_data, name):

    X_data = np.asarray(X_data, dtype=np.float32)
    y_data = np.asarray(y_data, dtype=np.float32)

    if len(X_data) == 0:
        print(f"No data for {name}")
        return None, None

    X_df = pd.DataFrame(X_data, columns=FEATURE_COLUMNS)

    maes = []
    models = []

    print(f"\n--- Training {name} ---")

    for fold, (tr, va) in enumerate(walk_forward_splits(len(X_df))):

        model = lgb.LGBMRegressor(
            n_estimators=800,
            max_depth=6,
            learning_rate=0.03,
            subsample=0.8,
            colsample_bytree=0.8,
            min_child_samples=30,
            objective="huber",
            alpha=0.9,
            random_state=42
        )

        model.fit(
            X_df.iloc[tr], y_data[tr],
            eval_set=[(X_df.iloc[va], y_data[va])],
            eval_metric="l1",
            callbacks=[lgb.early_stopping(50, verbose=False)],
            feature_name=FEATURE_COLUMNS
        )

        preds = model.predict(X_df.iloc[va])
        mae = mean_absolute_error(y_data[va], preds)

        print(f"{name} Fold {fold} MAE: {mae:.4f}")
        maes.append(mae)
        models.append(model)

    print(f"{name} Mean MAE: {np.mean(maes):.4f}")
    print(f"{name} Target std: {np.std(y_data):.4f}")
    print(f"{name} MAE/STD: {np.mean(maes)/np.std(y_data):.4f}")


    # Evaluate top percentiles using the last model trained
    if models:
        preds = models[-1].predict(X_df.iloc[va]) # Using 'va' from the last fold
        actual = y_data[va]
        for q in [50, 75, 90]:
            idx = preds >= np.percentile(preds, q)
            if np.sum(idx) > 0:
                print(f"{name} top {100-q}% mean actual: {actual[idx].mean():.4f}")
            else:
                print(f"{name} top {100-q}% mean actual: N/A (no predictions in this percentile)")

    return maes, models


# Train and evaluate models for each category
maes_up_low, models_up_low_60 = train_and_evaluate_model(X_up_low, y_up_low, "UP LOW")
maes_up_mid, models_up_mid_60 = train_and_evaluate_model(X_up_mid, y_up_mid, "UP MID")
maes_up_high, models_up_high_60 = train_and_evaluate_model(X_up_high, y_up_high, "UP HIGH")

maes_down_low, models_down_low_60 = train_and_evaluate_model(X_down_low, y_down_low, "DOWN LOW")
maes_down_mid, models_down_mid_60 = train_and_evaluate_model(X_down_mid, y_down_mid, "DOWN MID")
maes_down_high, models_down_high_60 = train_and_evaluate_model(X_down_high, y_down_high, "DOWN HIGH")


--- Training UP LOW ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1789
[LightGBM] [Info] Number of data points in the train set: 164816, number of used features: 9
[LightGBM] [Info] Start training from score 3.289425
UP LOW Fold 0 MAE: 0.3946
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1789
[LightGBM] [Info] Number of data points in the train set: 329632, number of used features: 9
[LightGBM] [Info] Start training from score 3.230899
UP LOW Fold 1 MAE: 0.4728
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012785 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [I

In [ ]:
path7="models/models_up_low_60.pkl"
joblib.dump(models_up_low_60, path7)
path8="models/models_up_mid_60.pkl"
joblib.dump(models_up_mid_60, path8)
path9="models/models_up_high_60.pkl"
joblib.dump(models_up_high_60, path9)
path10="models/models_down_low_60.pkl"
joblib.dump(models_down_low_60, path10)
path11="models/models_down_mid_60.pkl"
joblib.dump(models_down_mid_60, path11)
path12="models/models_down_high_60.pkl"
joblib.dump(models_down_high_60, path12)


['models/models_down_high_60.pkl']

In [ ]:
from google.colab import files
files.download("models/models_up_low_60.pkl")
files.download("models/models_up_mid_60.pkl")
files.download("models/models_up_high_60.pkl")
files.download("models/models_down_low_60.pkl")
files.download("models/models_down_mid_60.pkl")
files.download("models/models_down_high_60.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>